In [1]:
!pip install unsloth transformers scikit-learn accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
import pandas as pd

In [3]:
from datasets import load_dataset
dataset= load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
train_ds= pd.DataFrame(dataset["train"])
test_ds= pd.DataFrame(dataset["test"])

# Baseline model to check and compare both LG and our finetuned Bert model performance on this imdb dataset for our classification task

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [7]:
X_train= train_ds["text"]
y_train= train_ds["label"]
X_test= test_ds["text"]
y_test= test_ds["label"]

In [8]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)


In [9]:
lg= LogisticRegression()
lg.fit(X_train_tfidf,y_train)


LogisticRegression()

In [10]:
y_prediction=lg.predict(X_test_tfidf)

In [11]:
report= classification_report(y_test,y_prediction)

In [12]:
print(report)

              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



Finetuning Distillbert With Lora Technique (Parameter Efficient Finetuning)

In [13]:
# Exploring Qlora and lora finetuning methods more as well for parameter efficient finetuning
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, TaskType, LoraConfig

In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [15]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

will create a tokenizer function so that it will take the dataset from HF in batches to tokenize it and then feed to the model

In [16]:
def tokenize(batch):
  return(tokenizer(batch["text"],padding= True, max_length=256,truncation=True))

In [24]:
model= AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
tokenized_data = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [26]:
for name, module in model.named_modules():  # for checking the target modules in the models where lora adapter to be feed
    print(name)



distilbert
distilbert.embeddings
distilbert.embeddings.word_embeddings
distilbert.embeddings.position_embeddings
distilbert.embeddings.LayerNorm
distilbert.embeddings.dropout
distilbert.transformer
distilbert.transformer.layer
distilbert.transformer.layer.0
distilbert.transformer.layer.0.attention
distilbert.transformer.layer.0.attention.dropout
distilbert.transformer.layer.0.attention.q_lin
distilbert.transformer.layer.0.attention.k_lin
distilbert.transformer.layer.0.attention.v_lin
distilbert.transformer.layer.0.attention.out_lin
distilbert.transformer.layer.0.sa_layer_norm
distilbert.transformer.layer.0.ffn
distilbert.transformer.layer.0.ffn.dropout
distilbert.transformer.layer.0.ffn.lin1
distilbert.transformer.layer.0.ffn.lin2
distilbert.transformer.layer.0.ffn.activation
distilbert.transformer.layer.0.output_layer_norm
distilbert.transformer.layer.1
distilbert.transformer.layer.1.attention
distilbert.transformer.layer.1.attention.dropout
distilbert.transformer.layer.1.attention.q

In [27]:
config = LoraConfig(task_type=TaskType.SEQ_CLS,r=8,lora_alpha=32,lora_dropout=0.04,target_modules=['q_lin','k_lin,v_lin', 'o_lin'])


In [28]:
peft_model=get_peft_model(model,config)

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_score": f1_score(labels, preds)
    }


In [37]:
training_arguments= TrainingArguments(
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
)

Finetuning bert (Training)

In [39]:
trainer = Trainer(
    model=peft_model,
    args=training_arguments,
    train_dataset=tokenized_data["train"].shuffle(seed=20),  # subset for Colab speed
    eval_dataset=tokenized_data["test"].shuffle(seed=20),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-693712146.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [40]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tasmayu-swain (tasmayu-swain-maharaja-agrasen-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.325700,0.317848,0.868640,0.867911


TrainOutput(global_step=3125, training_loss=0.37908251220703126, metrics={'train_runtime': 678.4203, 'train_samples_per_second': 36.85, 'train_steps_per_second': 4.606, 'total_flos': 1681411430400000.0, 'train_loss': 0.37908251220703126, 'epoch': 1.0})

In [43]:
# set epoch 1 because it will take a lot of time
output = trainer.evaluate()
print(output)


{'eval_loss': 0.3178478479385376, 'eval_accuracy': 0.86864, 'eval_f1_score': 0.867910867991312, 'eval_runtime': 196.2155, 'eval_samples_per_second': 127.411, 'eval_steps_per_second': 15.926, 'epoch': 1.0}


MODEL INFERENCE

In [44]:
from transformers import pipeline
sentiment_pipeline_bert= pipeline("sentiment-analysis", model= peft_model, tokenizer=tokenizer)


Device set to use cuda:0


In [47]:
# label 0 for negative and label 1 for positive
print(sentiment_pipeline_bert("I loved the movie.It was Fantastic"))
print(sentiment_pipeline_bert("The movie was a disaster. i hope i will not see a movie like this in the future"))

[{'label': 'LABEL_1', 'score': 0.9846301674842834}]
[{'label': 'LABEL_0', 'score': 0.9116864800453186}]
